In [1]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time
import numpy as np
import json
from itertools import combinations_with_replacement
import json
from tqdm import tqdm
import argparse

In [2]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

prompt = '"Can you suggest some books similar to \"To Kill a Mockingbird\"?"'
advertisers = ['Apple', 'Amazon', 'Costco', 'Starbucks',]

ads = ["Experience innovation and elegance with Apple, a global leader in technology that continually sets the standard for user-friendly interfaces, stunning designs, and groundbreaking advancements. From the iPhone's captivating visuals and camera technology to the seamless synchronization of the Apple ecosystem comprising iPads, MacBooks, and Apple Watches, Apple products are designed to enhance your life professionally and personally. Whether you're a creative professional, a student, or someone who enjoys cutting-edge technology, Apple offers a range of devices that combine sophisticated functionality with exclusive features like Face ID, Siri, and Apple Pay, all secured with industry-leading privacy protection. Embrace a world where technology meets simplicity and sophistication with Apple.",
 'Discover the vast and vibrant world of Amazon, your one-stop shop for everything from the latest technology and trendy fashion to everyday essentials and unique finds. With Amazon, experience seamless shopping with fast, reliable delivery, and a hassle-free return policy that ensures your satisfaction. Immerse yourself in customer-centric innovation with Amazon Prime, offering exclusive benefits like free shipping, entertainment streaming, and much more. From pioneering in global ecommerce to pioneering sustainability initiatives, Amazon is not just a marketplace but a community built on trust and customer satisfaction. Shop at Amazon today and be a part of a shopping revolution!',
 "Experience the joy of shopping at Costco, where quality meets value in a dynamic retail environment tailored for your satisfaction. At Costco, members enjoy exclusive access to a vast selection of premium, bulk-sized products, from fresh groceries to high-tech electronics, all at unbeatably low prices. With a commitment to customer happiness, sustainability, and community support, Costco isn't just a shopping destination — it's a part of your community. Dive into a world of savings and discover why millions choose Costco as their trusted shopping partner. Join us today and see the difference Costco can make in your shopping experience, where every visit is more than just shopping — it's an adventure!",
 "Experience the warmth and delight of Starbucks, where every sip offers an invitation to a world of exquisite flavors and aromas. Renowned globally for its high-quality, handcrafted beverages, Starbucks is committed to sourcing the finest Arabica beans, expertly blending them into a variety of rich espressos, frothy cappuccinos, and creamy lattes. Each visit to a Starbucks store is more than just a coffee run—it's an opportunity to savor a moment of luxury amid the hustle of daily life. Whether you seek the comfort of a familiar classic or the thrill of a new seasonal specialty, Starbucks welcomes all to gather, connect, and enjoy a cup perfectly tailored to your taste. Step into your local Starbucks today and join us in celebrating the art of coffee."]

bids = np.array([3, 3, 2, 2,])

prompt_for_adv = 'Give me a paragraph about {} so that I can use it to advertise this brand.'

In [42]:
methods = ['single_alloc_with_replacement', 'single_alloc_without_replacement', 'single_alloc_naive_i', 'single_alloc_naive_ii']
num_of_repeat = 100
method_id = 0
dependent = 0

In [79]:
SOCIAL_WELFARE = 'social_welfare'
REVENUE = 'revenue'
RELEVANCE = 'relevance'
OUTPUT = 'output'
WINNER = 'winner'
RELEVANCE_TO_ORIGINAL = 'relevance_to_original'
SEGMENT_RELEVANCE_TO_ORIGINAL = "segment_relevence_to_original"
OUTPUT_MODIFIED = 'output_modified'

In [5]:
try:
    with open(path_file, 'r') as f:
        all_metrics = json.loads(f.read())
except:
    all_metrics = []

### Generate Original Answers for Scenario 0

In [9]:
def query_to_chatgpt(prompt):
    chat_completion_det = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        model = "gpt-4-turbo",
        logprobs = False,
        temperature = 1,
        max_tokens = 300
    )
    return chat_completion_det.choices[0].message.content

In [11]:
query = f'''
{prompt}
Answer this question for only 2 sentence.
'''

In [12]:
query_to_chatgpt(query)

'Sure! You might enjoy "The Help" by Kathryn Stockett, which explores themes of race and social justice in the South, much like "To Kill a Mockingbird." Another recommendation is "A Time to Kill" by John Grisham, which also tackles issues of racism and legal ethics in the Southern United States.'

In [13]:
orig_y = []
for i in range(10):
    orig_y.append(query_to_chatgpt(query))
    time.sleep(0.5)

In [15]:
with open('scenario0/orig_y.json', 'w') as f:
    f.write(json.dumps(orig_y, indent=4))

In [19]:
def relevance_to_original_output(original_output: list, outputs_with_ad: list):
    orig_embedding = model.encode(original_output)
    ads_embedding = model.encode(outputs_with_ad)
    return (1 + util.dot_score(ads_embedding, orig_embedding, ).numpy()) / 2

In [51]:
def add_relevance_to_output_to_metrics(path_file: str, use_output_modified: bool=False):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
        
    print(f'number of runs: {len(all_metrics)}')

    for metric in tqdm(all_metrics):
        if use_output_modified:
            outputs = metric[OUTPUT_MODIFIED]
        else:
            outputs = metric[OUTPUT]
        
        rels = relevance_to_original_output(original_output=orig_y, outputs_with_ad=outputs)
        rels = np.mean(rels, axis=1)
        rels = [float(x) for x in rels]
        
        metric[RELEVANCE_TO_ORIGINAL] = rels
    
    
    st = []
    for metric in all_metrics:
        st.append(metric[RELEVANCE_TO_ORIGINAL])

    st = np.array(st)
    result = np.mean(st, axis=0)
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
        
    return f'k=1: {result[0]:.3f}\nk=2: {result[1]:.3f}\nk=3: {result[2]:.3f}', result


In [44]:
def fix_outputs_naive_i(path_file: str):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
    
    print(f'number of runs: {len(all_metrics)}')
    
    sample_starts = []
    for i in range(10):
        sample_starts.append(query_to_chatgpt(query))
    
    for metric in tqdm(all_metrics):
        sample_id = np.random.randint(0, 10)
        sample_start = sample_starts[sample_id]
        metric[OUTPUT_MODIFIED] = [sample_start + metric[OUTPUT][i] for i in range(len(metric[OUTPUT]))]
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
        

In [46]:
method_id = 2
fix_outputs_naive_i(f'scenario0/{methods[method_id]}_dep{0}.json')

number of runs: 100


100%|██████████| 100/100 [00:00<00:00, 105703.23it/s]


In [53]:
all_res = []

### Average of Relevance to Original Output

When having $k=1$, $k=2$, and $k=3$ advertisement segments.

#### Single Allocation with Replcement

In [54]:
method_id = 0
out, t = add_relevance_to_output_to_metrics(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(t)

number of runs: 97


100%|██████████| 97/97 [00:01<00:00, 56.75it/s]

k=1: 0.727
k=2: 0.701
k=3: 0.686


#### Single Allocation without Replcement

In [56]:
method_id = 1
out, t = add_relevance_to_output_to_metrics(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(t)

number of runs: 40


100%|██████████| 40/40 [00:00<00:00, 47.48it/s]

k=1: 0.722
k=2: 0.693
k=3: 0.685


#### Single Allocation Naive (i)

without adding original answer to the beginning.

In [57]:
method_id = 2
out, t = add_relevance_to_output_to_metrics(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(t)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 46.18it/s]

k=1: 0.505
k=2: 0.504
k=3: 0.494


when adding original answer to the beginning.

In [58]:
method_id = 2
out, t = add_relevance_to_output_to_metrics(f'scenario0/{methods[method_id]}_dep{0}.json', use_output_modified=True)
print(out)
all_res.append(t)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 43.45it/s]

k=1: 0.776
k=2: 0.759
k=3: 0.692


#### Single Allocation Naive (ii)

In [59]:
method_id = 3
out, t = add_relevance_to_output_to_metrics(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(t)

number of runs: 80


100%|██████████| 80/80 [00:01<00:00, 57.52it/s]

k=1: 0.728
k=2: 0.696
k=3: 0.684


In [63]:
all_res = np.stack(all_res)

array([[0.72719562, 0.72219328, 0.50454816, 0.77601952, 0.728488  ],
       [0.70114769, 0.69329662, 0.5038823 , 0.75943102, 0.69550697],
       [0.686482  , 0.68545638, 0.49436125, 0.69243806, 0.68448711]])

In [78]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{x:.3f}' for x in all_res[:, 0]],
    'k=2': [f'{x:.3f}' for x in all_res[:, 1]],
    'k=3': [f'{x:.3f}' for x in all_res[:, 2]]
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.727,0.701,0.686
1,without Replacement,0.722,0.693,0.685
2,Naive (i) (wo output),0.505,0.504,0.494
3,Naive (i) (w output),0.776,0.759,0.692
4,Naive (ii),0.728,0.696,0.684


### Segment-wise Relevency

In this part, in order to get the similarity of the whole output to the original outputs, we measure the relevancy of each individual segment to those and take the average.

In [83]:
def segment_wise_relevancy(path_file: str, use_output_modified: bool=False):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
        
    print(f'number of runs: {len(all_metrics)}')

    for metric in tqdm(all_metrics):
        if use_output_modified:
            outputs = metric[OUTPUT_MODIFIED]
        else:
            outputs = metric[OUTPUT]
        
        segments = []
        for k in range(3):
            if k == 0:
                segments.append(outputs[0])
            else:
                prv_len = len(outputs[k - 1])
                segments.append(outputs[k][prv_len:])
        
        
        rels = relevance_to_original_output(original_output=orig_y, outputs_with_ad=segments)
        rels = np.mean(rels, axis=1)
        rels = [float(x) for x in rels]
        
        metric[SEGMENT_RELEVANCE_TO_ORIGINAL] = rels
    
    
    st = []
    for metric in all_metrics:
        st.append(metric[SEGMENT_RELEVANCE_TO_ORIGINAL])

    st = np.array(st)
    result = np.mean(st, axis=0)
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
        
    return f'k=1: {result[0]:.3f}\nk=2: {result[1]:.3f}\nk=3: {result[2]:.3f}', result


In [94]:
segments_res = []

In [95]:
method_id = 0
out, t = segment_wise_relevancy(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(t)

number of runs: 97


100%|██████████| 97/97 [00:01<00:00, 55.88it/s]


k=1: 0.727
k=2: 0.577
k=3: 0.569


In [96]:
method_id = 1
out, t = segment_wise_relevancy(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(t)

number of runs: 40


100%|██████████| 40/40 [00:00<00:00, 54.08it/s]

k=1: 0.722
k=2: 0.577
k=3: 0.576


In [97]:
method_id = 2
out, t = segment_wise_relevancy(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(t)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 55.06it/s]

k=1: 0.505
k=2: 0.504
k=3: 0.503


In [98]:
method_id = 2
out, t = segment_wise_relevancy(f'scenario0/{methods[method_id]}_dep{0}.json', use_output_modified=True)
print(out)
segments_res.append(t)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 49.42it/s]

k=1: 0.776
k=2: 0.504
k=3: 0.503


In [99]:
method_id = 3
out, t = segment_wise_relevancy(f'scenario0/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(t)

number of runs: 80


100%|██████████| 80/80 [00:01<00:00, 60.11it/s]

k=1: 0.728
k=2: 0.571
k=3: 0.580


In [100]:
np_segments_res = np.stack(segments_res)

In [101]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{x:.3f}' for x in np_segments_res[:, 0]],
    'k=2': [f'{x:.3f}' for x in np_segments_res[:, 1]],
    'k=3': [f'{x:.3f}' for x in np_segments_res[:, 2]]
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.727,0.577,0.569
1,without Replacement,0.722,0.577,0.576
2,Naive (i) (wo output),0.505,0.504,0.503
3,Naive (i) (w output),0.776,0.504,0.503
4,Naive (ii),0.728,0.571,0.580


In [104]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k<=1': [f'{x:.3f}' for x in np_segments_res[:, 0]],
    'k<=2': [f'{x:.3f}' for x in np.mean(np_segments_res[:, :2], axis=1)],
    'k<=3': [f'{x:.3f}' for x in np.mean(np_segments_res[:, :3], axis=1)],
})
df

,Mechanism,k<=1,k<=2,k<=3
0,with Replacement,0.727,0.652,0.625
1,without Replacement,0.722,0.649,0.625
2,Naive (i) (wo output),0.505,0.504,0.504
3,Naive (i) (w output),0.776,0.640,0.594
4,Naive (ii),0.728,0.650,0.626


In [105]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'Average over k=[1, 2, 3]': [f'{x:.3f}' for x in np.mean(np_segments_res, axis=1)],
})
df

,Mechanism,"Average over k=[1, 2, 3]"
0,with Replacement,0.625
1,without Replacement,0.625
2,Naive (i) (wo output),0.504
3,Naive (i) (w output),0.594
4,Naive (ii),0.626
